What kind of Data?

|Better              |Worse|
|---------------------|----------|
|Higher Quality       |Lower Quality|
|Diversity            |Homogeneity|
|Real                 |Generated|
|More                 |Less|





Steps tp Prepare your data

1. Collect instruction-response pairs

2. Concatenate pairs (add prompt template, if applicable)

3. Tokenize: Pad, Truncate

4. Split into Train/Test

Tokenizing your data :- Tokenize the data means converting text data to number that represent each pieces of texts. 

It not necessary by words, its based on frequency of, common character occurances. ING token

  Fine  Tun     ing  is   fun  for  all  !
                                                Encoding
[34389, 13932, 278, 318, 1257, 329, 477, 0]
                                                Decoding
Fine tuning is fun for all

So every single, verb in gerund, you know, fine-tuning or tokenizing all has ING and that maps onto the token 278 here.

When you decode it with the help of same tokenizer it turns back into the same text.

There are lot of Tokenizers and a tokenizer is really associated with a specific model for each model as it was trained on it,

And if you give the wrong tokenizer to your model, it'll be very confused because it will expect different numbers to represent different sets of letters and different words


There are multiple popular tokenizers:

1. Use the tokenizer associated with your model!

In [1]:
import pandas as pd
import datasets

from pprint import pprint
from transformers import AutoTokenizer      # Most important. (It automatically finds the right tokenizer or for your model when you specify what model is.)

/Users/apple/anaconda3/envs/enve/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")

In [3]:
text = "Hi, how are you?"

In [4]:
encoded_text = tokenizer(text)["input_ids"]

In [5]:
encoded_text

[12764, 13, 849, 403, 368, 32]

In [6]:
decoded_text = tokenizer.decode(encoded_text)
print("Decoded tokens back into text: ", decoded_text)

Decoded tokens back into text:  Hi, how are you?


In [7]:
list_texts = ["Hi, How are you?", "I'm good" ,"YES"]
encoded_texts = tokenizer(list_text)
print("Encoded several texts: ", encoded_texts["input_ids"])

NameError: name 'list_text' is not defined

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
encoded_texts_longest = tokenizer(list_texts,padding=True)
print("Encoded Serval texts: ",encoded_texts_longest["input_ids"])

Encoded Serval texts:  [[12764, 13, 1359, 403, 368, 32], [42, 1353, 1175, 0, 0, 0], [24239, 0, 0, 0, 0, 0]]


Truncation is technique of making encoded text much shorter as you model will also have max_length that it can handle and take in so it can't just fit 

everything in, their is the limit to prompt length and so this is the same thing and truncation is strategy to handle making those encoded text much shorter

and that fit actually into the model so this is one way to make it shorter.

In [ ]:
encoded_texts_truncation = tokenizer(list_texts, max_length=3, truncation=True)  #by default will count from right, so first three will be there and will neglect rest.
print("using truncation: ", encoded_texts_truncation["input_ids"])

using truncation:  [[12764, 13, 1359], [42, 1353, 1175], [24239]]


In [ ]:
tokenizer.truncation_side = "left"
encoded_texts_truncation_left = tokenizer(list_texts, max_length=3, truncation=True)
print("Encoded Serval texts: ",encoded_texts_truncation_left["input_ids"])

Encoded Serval texts:  [[403, 368, 32], [42, 1353, 1175], [24239]]


In [ ]:
encoded_texts_both = tokenizer(list_texts, max_length=3, truncation=True, padding=True)
print("using both padding and truncation: ", encoded_texts_both["input_ids"])

using both padding and truncation:  [[403, 368, 32], [42, 1353, 1175], [24239, 0, 0]]


In [9]:
filename ="/Users/apple/Desktop/project/LLM/Finetuning_llm/Data/lamini_docs.jsonl"
instruction_dataset_df = pd.read_json(filename, lines=True)

In [22]:
examples = instruction_dataset_df.to_dict()

if "question" in examples and "answer" in examples:
    text = examples["question"][0] + examples["answer"][0]
elif "instruction" in examples and "response" in examples:
    text = examples["instruction"][0] + examples["response"][0]
elif "input" in examples and "output" in examples:
    text = examples["input"][0] + examples["output"][0]
else:
    text = examples["text"][0]


In [23]:
prompt_template = """### Question: 
{question}

### Answeer:"""

In [24]:
num_examples = len(examples["question"])
finetuning_dataset =[]

for i in range(num_examples):
    question = examples["question"][i]
    answer = examples["answer"][i]
    text_with_prompt_template = prompt_template.format(question=question)
    finetuning_dataset.append({"question":text_with_prompt_template, "answer":answer})
    
print("One datapoint in the finetuning dataset: ")
pprint(finetuning_dataset[0])

One datapoint in the finetuning dataset: 
{'answer': 'Lamini has documentation on Getting Started, Authentication, '
           'Question Answer Model, Python Library, Batching, Error Handling, '
           'Advanced topics, and class documentation on LLM Engine available '
           'at https://lamini-ai.github.io/.',
 'question': '### Question: \n'
             'What are the different types of documents available in the '
             'repository (e.g., installation guide, API documentation, '
             "developer's guide)?\n"
             '\n'
             '### Answeer:'}


Now run tokenizer

In [26]:
tokenizer.pad_token = tokenizer.eos_token
text = finetuning_dataset[0]["question"] + finetuning_dataset[0]["answer"]
tokenized_inputs = tokenizer(
    text,
    return_tensors="np",
    padding=True
)
print(tokenized_inputs["input_ids"])

[[ 4118 19782    27   209   187  1276   403   253  1027  3510   273  7177
   2130   275   253 18491   313    70    15    72   904 12692  7102    13
   8990 10097    13 13722   434  7102  6177   187   187  4118 42259   664
    254    27    45  4988    74   556 10097   327 27669 11075   264    13
   5271 23058    13 19782 37741 10031    13 13814 11397    13   378 16464
     13 11759 10535  1981    13 21798 12989    13   285   966 10097   327
  21708    46 10797  2130   387  5987  1358    77  4988    74    14  2284
     15  7280    15   900 14206]]


I used padding because , don't know how long these tokens actually will be, and so what's important is that I figure out, the minimum between the max length 

nd tokenized inputs. you can always just pad to the longest, you can always pad to the max length.

In [27]:
max_length = 2048
max_length = min(
    tokenized_inputs["input_ids"].shape[1],
    max_length
)

In [29]:
tokenized_inputs = tokenizer(
    text,
    return_tensors = "np",
    truncation=True,
    max_length=max_length
)

In [30]:
tokenized_inputs["input_ids"]

array([[ 4118, 19782,    27,   209,   187,  1276,   403,   253,  1027,
         3510,   273,  7177,  2130,   275,   253, 18491,   313,    70,
           15,    72,   904, 12692,  7102,    13,  8990, 10097,    13,
        13722,   434,  7102,  6177,   187,   187,  4118, 42259,   664,
          254,    27,    45,  4988,    74,   556, 10097,   327, 27669,
        11075,   264,    13,  5271, 23058,    13, 19782, 37741, 10031,
           13, 13814, 11397,    13,   378, 16464,    13, 11759, 10535,
         1981,    13, 21798, 12989,    13,   285,   966, 10097,   327,
        21708,    46, 10797,  2130,   387,  5987,  1358,    77,  4988,
           74,    14,  2284,    15,  7280,    15,   900, 14206]])

In [31]:
def tokenize_function(examples):
    if "question" in examples and "answer" in examples:
        text = examples["question"][0] + examples["answer"][0]
    elif "instruction" in examples and "response" in examples:
        text = examples["instruction"][0] + examples["response"][0]
    elif "input" in examples and "output" in examples:
        text = examples["input"][0] + examples["output"][0]
    else:
        text = examples["text"][0]
    
    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        return_tensors = "np",
        padding=True
    )
    
    max_length = min(
        tokenized_inputs["input_ids"].shape[1],
        2048
    )
    
    tokenizer.truncation_side="left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors = "np",
        truncation = True,
        max_length = max_length
    )
    
    return tokenized_inputs

In [35]:
finetuning_dataset_loaded = datasets.load_dataset("json",data_files=filename,split = "train")

tokenized_dataset = finetuning_dataset_loaded.map(
    tokenize_function,
    batched = True,
    batch_size = 1,
    drop_last_batch=True
)

print(tokenized_dataset)

Map: 100%|██████████| 1400/1400 [00:01<00:00, 1267.64 examples/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 1400
})


In [36]:
tokenized_dataset = tokenized_dataset.add_column("labels",tokenized_dataset["input_ids"])

In [37]:
split_dataset = tokenized_dataset.train_test_split(test_size = 0.1, shuffle=True, seed = 123)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
})
